In [13]:
%pip install langchain==0.1.11

Note: you may need to restart the kernel to use updated packages.


# this is a demo notebook to introduce basic langchain abilities

## in this notebook, we will be playing around with langchain for building nerdoscope - a basic horoscope generator for nerds 🤓

In [18]:
# Make sure you pull a model from ollama using `ollama pull <model-name>` (e.g., phi)
from langchain_community.llms import Ollama

llm = Ollama(model="orca-mini:3b")

In [19]:
# Sanity check: Say hey to your local LLM!

llm.invoke("Hey!")

' Hello! How can I assist you today?'

In [20]:
# Naively ask for a nerdoscope reading

zodiac_role = "🧑‍💻 Software Develpoer"
query = f'Please generate a funny "nerdoscope" (horoscope for nerds) reading for a {zodiac_role}. Keep the answer short, 5 sentences max.'

for chunks in llm.stream(query):
    print(chunks, end="", flush=True)

 Your Nerdoscope Reading for the week is: You'll be able to code your way to success, but don't forget to take a break and read a good book now and then.

In [27]:
# Let's improve this a bit by using system prompt as context
# This helps the model understand the task better

from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a horoscope bot, that provides a "nerdoscope" (horoscope for nerds) reading for a personnas of the tech industry.'
Given the user's role (as their zodiac sign), create a nerdoscope readings that is funny and relatable to their role.
Keep the answer short, 5 sentences max.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{zodiac_role}")
])

# We'll now use the prompt and llm to create a chain, this is called LCEL (Langchain Expression Language)
chain = prompt | llm

zodiac_role = "🧑‍💻 Software Develpoer"
for s in chain.stream({"zodiac_role": zodiac_role}):
    print(s, end="", flush=True)

 Your nerdoscope reading for the tech industry is as follows: You have a knack for solving complex problems with ease, and you're always up for a good coding challenge. However, sometimes your mind can be a bit too analytical, which can make it hard for you to relax and enjoy the fun aspects of life. Don't forget to take breaks and prioritize self-care!

In [49]:
# Nice! Let's organize this into a function

from langchain_core.runnables.base import Runnable


def stream_print(chain, context):
    # This function will stream the output of the chain and print it in a readable way for the notebook
    word_count = 0
    buffer = ""

    for s in chain.stream(context):
        buffer += s
        words = buffer.rsplit(' ', 1)
        if len(words) > 1:
            for word in words[:-1]:
                print(word, end=' ', flush=True)
                word_count += 1
                if word_count == 10:
                    print()
                    word_count = 0
            buffer = words[-1]
    if buffer:
        print(buffer, end='', flush=True)

In [48]:
system_prompt = """
You are a horoscope bot for personnas in the software development industry.
Given a user's role, generate a personalized "nerdoscope" (horoscope for nerds) message for the week.
Guidelines for the message:
* The message should consists of a single paragraph, 5 sentences max.
* The message should be in the spirit of a horoscope (refer to it as nerdoscope), offering a blend of foresight and advice relevant to the user role, but in a humorous way.
* If possible, the foresight and/or advice should reference a popular tool or technology in the software development industry, relevant to the role.
* The tone should be fun and lighthearted.
* The message should be funny and sarcastic, and personalized to the role.
* The message should include technical jargon.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{zodiac_role}")
])

chain = prompt | llm

zodiac_role = "🧑‍💻 Software Develpoer"
stream_print(chain, {"zodiac_role": zodiac_role})

 Hey there! This week's nerdoscope says that you'll have 
an unexpected encounter with a programming language that you can't 
even begin to understand. Don't worry, though - use your 
wits and technical expertise to decipher the code and uncover 
the hidden message that lies within. Remember, chaos is the 
key to unlocking the secrets of software development!

In [ ]:
# I think we are ready to create our app :)